[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/stable-diffusion-webui-colab/blob/training/simple_dreambooth_trainer.ipynb)

In [ ]:
#@title Train DreamBooth
from google.colab import drive
drive.mount('/content/drive')

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!apt -y install -qq aria2
!pip install -q --pre xformers==0.0.17.dev476 -U
!pip install -q --pre triton

!pip install -q diffusers==0.13.1 -U
!pip install -q transformers==4.26.1 -U
!pip install -q accelerate==0.16.0 -U
!pip install -q ftfy==6.1.1 -U
!pip install -q bitsandbytes==0.37.0 -U
!pip install -q safetensors==0.2.8 -U
!pip install -q huggingface-hub==0.12.1 -U

!git clone https://huggingface.co/JosephusCheung/ACertainty /content/model
!mv /content/model/ACertainty.ckpt /content/model/model.ckpt

!mkdir /content/trained
!mkdir /content/trainer
!mkdir /content/trainer/diffusers
!mkdir /content/trainer/diffusers/dreambooth
!mkdir /content/trainer/diffusers/dreambooth/class_data_dir

!wget https://raw.githubusercontent.com/camenduru/trainer/main/diffusers/dreambooth/train_dreambooth.py -O /content/trainer/diffusers/dreambooth/train_dreambooth.py
!wget https://raw.githubusercontent.com/camenduru/trainer/main/diffusers/dreambooth/convert_diffusers_to_original_stable_diffusion.py -O /content/trainer/diffusers/dreambooth/convert_diffusers_to_original_stable_diffusion.py


root_path = "/content/drive/MyDrive/AI/training" #@param {type:"string"}
instance_prompt = "parkminyoung" #@param {type:"string"}
class_prompt = "person" #@param {type:"string"}
images_path = f"{root_path}/{instance_prompt}"
output_dir = f"/content/{instance_prompt}"
max_train_steps = 650 #@param {type:"integer"}

!python -u /content/trainer/diffusers/dreambooth/train_dreambooth.py \
                    --pretrained_model_name_or_path="/content/model"  \
                    --instance_data_dir={images_path} \
                    --output_dir={output_dir} \
                    --learning_rate=5e-6 \
                    --max_train_steps={max_train_steps} \
                    --instance_prompt={instance_prompt} \
                    --resolution=512 \
                    --center_crop \
                    --train_batch_size=1 \
                    --gradient_accumulation_steps=1 \
                    --max_grad_norm=1.0 \
                    --mixed_precision="fp16" \
                    --gradient_checkpointing \
                    --enable_xformers_memory_efficient_attention \
                    --use_8bit_adam \
                    --with_prior_preservation \
                    --class_data_dir="/content/trainer/diffusers/dreambooth/class_data_dir" \
                    --prior_loss_weight=1.0 \
                    --sample_batch_size=2 \
                    --class_prompt={class_prompt} \
                    --seed=69 \
                    --num_class_images=12

In [ ]:
#@title Test Model
from diffusers import StableDiffusionPipeline
import gc, os, torch

instance_prompt = "parkminyoung" #@param {type:"string"}
class_prompt = "person" #@param {type:"string"}
output_dir = f"/content/{instance_prompt}"

load_model = True #@param {type:"boolean"}
if load_model:
  pipe = StableDiffusionPipeline.from_pretrained(output_dir, safety_checker=None, torch_dtype=torch.float16).to("cuda")
  pipe.enable_xformers_memory_efficient_attention()

root_folder = '/content/drive/MyDrive/AI/trained' #@param {type: 'string'}
image_folder = '001' #@param {type: 'string'}
if os.path.exists(f"{root_folder}") == False:
  os.mkdir(f"{root_folder}")
if os.path.exists(f"{root_folder}/{image_folder}") == False:
  os.mkdir(f"{root_folder}/{image_folder}")
name = max([int(f[:f.index('.')]) for f in os.listdir(f"{root_folder}/{image_folder}")], default=0)

prompt = "drawing" #@param {type: 'string'}
negative_prompt = "" #@param {type: 'string'}
num_inference_steps = 50 #@param {type: 'integer'}
guidance_scale = 7.5 #@param {type: 'raw'}
height = 512 #@param {type: 'integer'}
width = 512 #@param {type: 'integer'}

def generate(prompt, name):
  gc.collect()
  torch.cuda.empty_cache()
  image = pipe(prompt, negative_prompt=negative_prompt, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]
  image.save(f"{root_folder}/{image_folder}/{name:04}.png")
  display(image)

full = f"{instance_prompt} {class_prompt} {prompt}"
name += 1
generate(full, name)

In [ ]:
#@title Convert Diffusers to Original Stable Diffusion
instance_prompt = "parkminyoung" #@param {type:"string"}
output_dir = f"/content/{instance_prompt}"
ckpt_file = f"{output_dir}/{instance_prompt}.ckpt"
!python /content/trainer/diffusers/dreambooth/convert_diffusers_to_original_stable_diffusion.py --model_path /content/{instance_prompt} --checkpoint_path {ckpt_file} --half

In [ ]:
#@title Convert Diffusers to Original Stable Diffusion
import gc, os, torch
if 'pipe' in locals():
    del pipe
gc.collect()
torch.cuda.empty_cache()
instance_prompt = "parkminyoung" #@param {type:"string"}
output_dir = f"/content/{instance_prompt}"
ckpt_file = f"{output_dir}/{instance_prompt}.ckpt"
!python /content/trainer/diffusers/dreambooth/convert_diffusers_to_original_stable_diffusion.py --model_path /content/{instance_prompt} --checkpoint_path {ckpt_file} --half